# CM4105 Cell Counter

## Preliminaries

In [17]:
import yolov5
import wandb
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import os
import shutil
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [18]:
# Log into Weights & Biases for logging
wandb.login()

True

## Training

In [19]:
# Train a new model on the dataset
# !python train.py --project cm4105 --img 224 --batch 128 --epochs 300 --data 'dataset/data.yaml' --weights yolov5l.pt --cache

## Counting

In [20]:
cell_counts = []

In [21]:
# Load YOLOv5s
# model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

In [22]:
# Load custom model
model = yolov5.load('models/run7/best.pt')

In [23]:
'''
Run inference on the given image & return total cell count, first DataFrame, & first patch
'''
def count_cells(img: str, edge: int = 2) -> tuple([np.ndarray, pd.DataFrame, int]):
  patches = []

  # Run inference on all patches of the img passed
  for i in os.listdir('dataset/images/test'):
    if i[12:14] == img:
      patches.append(f'dataset/images/test/{i}')
  results = model(patches)

  # Results as DataFrames
  dfs = results.pandas().xyxy

  # Get the first DataFrame & image in the results
  df = dfs[0]
  results.render()
  img = results.imgs[0]

  # Count
  count = 0
  for df in dfs:
    # Remove cells along the top-left edge (default 2 px) of the image
    df = df.loc[df['xmin'] > edge]
    df = df.loc[df['ymin'] > edge]
    count += len(df)

  return df, img, count

In [24]:
df, img, cell_count = count_cells('34')

plt.imshow(img)
plt.show()
print(f'Count: {cell_count}')
cell_counts.append(cell_count)

KeyboardInterrupt: 

In [ ]:
df, img, cell_count = count_cells('39')

plt.imshow(img)
plt.show()
print(f'Count: {cell_count}')
cell_counts.append(cell_count)

In [ ]:
df, img, cell_count = count_cells(img='41')

plt.imshow(img)
plt.show()
print(f'Count: {cell_count}')
cell_counts.append(cell_count)

In [ ]:
df, img, cell_count = count_cells('44')

plt.imshow(img)
plt.show()
print(f'Count: {cell_count}')
cell_counts.append(cell_count)

In [ ]:
# Display counts
print(cell_counts)

## Evaluation

In [ ]:
# Ground Truth counts for test set images
images = [37, 39, 41, 44]
cell_counts_gt = [81, 58, 86, 102]

# Construct DataFrame
df = pd.DataFrame({
    'Image': images,
    'Ground Truth': cell_counts_gt,
    'Model Count': cell_counts
    })

# Calculate absolute & percentage error
df['Num. Error'] = abs(df['Ground Truth'] - df['Model Count'])
df['% Error'] = (df['Num. Error'] / df['Ground Truth']) * 100
# df.to_csv('cell_counts.csv')
df

In [ ]:
# print(df.to_latex())

### Graphing

In [ ]:
df: pd.DataFrame = pd.read_csv('counts_by_model.csv')
df

In [ ]:
fig = px.bar(df, x='Model', y='Num_Error', labels={'Num_Error': 'Num. Error'})
fig.show()
# fig.write_image("num_error.png")

# ax = df.plot.bar(x='Model', y='Num_Error')

In [ ]:
fig = px.bar(df, x='Model', y='Pct_Error', labels={'Pct_Error': 'Pct. Error'})
fig.show()
# fig.write_image("pct_error.png")

# ax = df.plot.bar(x='Model', y='Num_Error',)